In [18]:
import numpy as np
import matplotlib.pyplot as plt

# Load data from files (adjust paths as needed)
file1 = 'test.txt'
file2 = 'Vs-P.txt'

# Read the data from both files
data1 = np.loadtxt(file1, skiprows=1, comments='#')
data2 = np.loadtxt(file2, skiprows=1, comments='#')

# Extract columns from file 1 (T1: Temperature, P1: Pressure, Vs1: Shear wave velocity)
T1 = data1[:, 0]
P1 = data1[:, 1]
Vs1 = data1[:, 6]

# Extract columns from file 2 (Distance, Depth, Pressure, Vs)
Distance2 = data2[:, 0]
Depth2 = data2[:, 1]
P2 = data2[:, 2] / 10000  # Convert Pressure from bar to GPa
Vs2 = data2[:, 3]

# Filter out rows in file 2 with NaN Vs values
valid_indices = ~np.isnan(Vs2)
Distance2 = Distance2[valid_indices]
Depth2 = Depth2[valid_indices]
P2 = P2[valid_indices]
Vs2 = Vs2[valid_indices]

# Dist-method for bilinear interpolation
def dist_method_temp(T1, P1, Vs1, P2, Vs2):
    T_interpolated = []

    for pi, vsi in zip(P2, Vs2):
        # Find the surrounding points in file 1
        T_lower = T1[T1 <= pi].max() #if np.any(T1 <= pi) else np.nan  # Lower T bound
        T_upper = T1[T1 > pi].min() #if np.any(T1 > pi) else np.nan    # Upper T bound
        P_lower = P1[P1 <= pi].max() #if np.any(P1 <= pi) else np.nan  # Lower P bound
        P_upper = P1[P1 > pi].min() #if np.any(P1 > pi) else np.nan    # Upper P bound

        # Get corresponding Vs values for surrounding points
        if len(Vs1[(T1 == T_lower) & (P1 == P_lower)]) > 0:
            Vs_T1P1 = Vs1[(T1 == T_lower) & (P1 == P_lower)][0]
        else:
            Vs_T1P1 = np.nan  # Handle missing value

        if len(Vs1[(T1 == T_upper) & (P1 == P_lower)]) > 0:
            Vs_T2P1 = Vs1[(T1 == T_upper) & (P1 == P_lower)][0]
        else:
            Vs_T2P1 = np.nan

        if len(Vs1[(T1 == T_lower) & (P1 == P_upper)]) > 0:
            Vs_T1P2 = Vs1[(T1 == T_lower) & (P1 == P_upper)][0]
        else:
            Vs_T1P2 = np.nan

        if len(Vs1[(T1 == T_upper) & (P1 == P_upper)]) > 0:
            Vs_T2P2 = Vs1[(T1 == T_upper) & (P1 == P_upper)][0]
        else:
            Vs_T2P2 = np.nan

        # Check for NaN values before proceeding
        if np.isnan(Vs_T1P1) or np.isnan(Vs_T2P1) or np.isnan(Vs_T1P2) or np.isnan(Vs_T2P2):
            T_interpolated_value = np.nan  # If any value is missing, set result to NaN
        else:
            # Calculate weights for bilinear interpolation
            T2_T1 = T_upper - T_lower
            P2_P1 = P_upper - P_lower

            w_T1 = (T_upper - pi) / T2_T1 if T2_T1 != 0 else 0.5
            w_T2 = (pi - T_lower) / T2_T1 if T2_T1 != 0 else 0.5
            w_P1 = (P_upper - pi) / P2_P1 if P2_P1 != 0 else 0.5
            w_P2 = (pi - P_lower) / P2_P1 if P2_P1 != 0 else 0.5

            # Calculate the interpolated temperature value
            T_interpolated_value = (w_T1 * w_P1 * Vs_T1P1 +
                                    w_T2 * w_P1 * Vs_T2P1 +
                                    w_T1 * w_P2 * Vs_T1P2 +
                                    w_T2 * w_P2 * Vs_T2P2)

        T_interpolated.append(T_interpolated_value)

    return np.array(T_interpolated)


# Calculate interpolated temperatures for Vs and P from file 2
T_interpolated = dist_method_temp(T1, P1, Vs1, P2, Vs2)

# Convert temperatures from Kelvin to Celsius
T_interpolated_C = T_interpolated - 273.15

# Write final results to a new file
output_file = 'final_output.txt'
with open(output_file, 'w') as f:
    f.write('# Distance(Km)   Depth(Km)   Pressure(GPa)   Vs(Km/s)   Temp(oC)\n')
    for d, z, p, vs, temp in zip(Distance2, Depth2, P2, Vs2, T_interpolated_C):
        f.write(f'{d:10.6f}   {z:10.6f}   {p:10.6f}   {vs:10.6f}   {temp:10.6f}\n')

print(f'Final output written to {output_file}')

ValueError: zero-size array to reduction operation maximum which has no identity